## Quantization Basics

### What is quantization

- Changing floating point numbers from 32bits to lower precision, thereby reducing memory use (full precision to half precision)

### Types of quantization

- Quantization involves 2 parameters: a `scale` and a `zero point`
    - Symmetric Quantization
        - Imagine I have am array of 32-bit floating point values in range [0, 1000]. 
        - What happens if I want to convert these values into unsigned 8bit integers? (i.e. in range [0, 255])
        - Note: Understand how the floating point values are stored 
            - A 32-bit float are split into
                - 1 bit for sign
                - 8 bits for exponent
                - 23 bits for mantissa (fractional value)
            - A 16-bit float are split into
                - 1 bit for sign
                - 5 bits for exponent
                - 10 bits for mantissa (fractional value)

        - We scale the 32-bit value using min-max scaling
            - Compute the scaling factor: $\frac{1000-0}{255-0} = 3.92$
            - Then, for any 32-bit value $x$, convert to uint-8 by taking $x / 3.92$, and round it

        - Batch Norm is a type of symmetric quantization

    - Asymmetric Quantization
        - Imagine now I have am array of 32-bit floating point values in range [-20, 1000]
        - What happens if I want to convert these values into unsigned 8bit integers? (i.e. in range [0, 255])
        - We scale the 32-bit value using the same min-max scaling
            - Compute the scaling factor: $\frac{1000 - (-20)}{255-0} = 4$
            - Then, for any 32-bit value $x$, convert to uint-8 by taking $x / 4$, and round it
        
        - But now we have a problem. imagine my initial 32bit float is -20. After conversion, it becomes -5, still below the range of the desired value!

        - Hence, in such cases, a new parameter `zero point` is introduced and added post scaling, so the minimum value maps to 0. In this case, `zero point = 5`

### Post Training Quantization (PTQ)

- Suppose we have a pretrained model that's extremely large. W
- We want to use it, but the weights don't fit in memory. What's the next step?
- First, calibrate the model
    - Squeeze the range of the values into a smaller range (calibration)
    - Convert weights into lower precision (quantization)

### Quantization Aware Training (QAT)

- In post training quantization, there is inevitably data loss, and so this impacts accuracy
- QAT avoids this by adding a fine-tuning step after quantization. So take new training data and do a new round of `fit`
- This creates better outcomes for the models

## Quantization Techniques

- Let's say we have a pre-trained base model (say GPT4)
    - We can train all parameters based on the data we have, which we'll call "Full Parameter Fine Tuning"
    - OR we can fine tune the model for specific domains (e.g. sales chat bot, etc). We can call this "domain specific fine tuning"
    - OR we can fine tune the model for specific tasks (document retrieval bot, q&a bot etc). We can call this "task specific fine tuning"

- Full Parameter Fine Tuning
    - Suppose we have a base model with 175B weights
    - In this case, to perform full parameter fine tuning, every run needs you to compute 175B gradients. This is obviously horrible, because you probably don't have the hardware needed to do this
    - This is where `LORA` and `QLORA` come in

### LORA

- Low Rank Adaptation of Large Language Models
    - Suppose I have pretrained weights denoted by set $S$ 
    - Initialize a new set of weights $L$ with the same size as $S$. These are the LORA weights
    - $L$ will be updated, and the final weights applied for inference will be $L + S$

- Won't the same resource constraint with $S$ also be present in $L$ if the shapes are the same?
    - We can use matrix decomposition to simplify $L$;
    - So instead of, for example, one single $N \times N$ matrix with $N^2$ elements, we can have $1 \times N$ and $N \times 1$ with $2N$ elements

- Therefore, the LORA equation is simply $W_0 + \triangle W = W_0 + B \cdot A$, where $B \cdot A$ is simply the matrix decomposition of the LORA weight matrix

- Reduction in weights: Suppose we want to decompose an $m \times n$ matrix with 7B parameters into 2 matrices. For simplicity, assume $m=n$
    - For rank 1, the 2 matrices have dimensions $n \times 1$ and $1 \times n$. Therefore, $n = \sqrt{7B} \approx 83,000$, and total parameters get reduced to $83,000 * 2 \approx 167,000$
    - For rank 8, the 2 matrices have dimensions $n \times 8$ and $8 \times n$. Therefore, $n = 83,000$, and total parameters get reduced to $83,000 * 2 * 8 \approx 1M$

| Rank | 7B | 13B | 17B | 180B |
| --- | --- | --- | --- | --- |
| 1 | 83K * 2 | 114K * 2 | 130K * 2 | 424K * 2 |
| 8 | 83K * 2 | 114K * 2 | 130K * 2 | 424K * 2 |

In [5]:
180_000_000_000 ** 0.5

424264.0687119285

### QLORA

- Quantized Low Rank Adaptation of Large Language Models
- Instead of leaving the LORA weights as they are, reduce precision of LORA weights. That's all

## 1-Bit LLM

- Instead of 32bit precision float, or quantization to 8bit, we reduce the weights to ternary operators {-1,0,1}
- Why is this great? 
    - If $W \in [-1,0,1]$, then $W \cdot x$ reduces to $\sum_{w_i = 1} x - \sum_{w_i = -1} x$
    - No more matrix multiplication! Only summation is needed

## Open source models

- Llama
- Gemma